In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from tqdm import tqdm
from google.colab import files
import shutil
# from huggingface_hub import notebook_login
# login()

In [3]:
df = pd.read_csv('sciq.csv')
df.head()

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [5]:
len(df)

11679

In [4]:
df = df.dropna()
len(df)

10481

In [ ]:
# token_counts = df['support'].apply(lambda x: len(tokenizer.encode(str(x))))

# max_token_count = token_counts.max()
# max_token_count

In [7]:
df['support'][0], df['question'][0], df['correct_answer'][0], df['distractor1'][0], df['distractor2'][0], df['distractor3'][0]

('Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.',
 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'mesophilic organisms',
 'protozoa',
 'gymnosperms',
 'viruses')

In [5]:
print(tokenizer.encode(df['support'][0]))
print(tokenizer.encode(df['question'][0]))
print(tokenizer.encode(df['correct_answer'][0]))


[44, 274, 2522, 2915, 1663, 1266, 287, 10768, 5951, 11, 6032, 1022, 1679, 7200, 34, 290, 2319, 7200, 34, 357, 3324, 7200, 37, 290, 14436, 7200, 37, 737, 14937, 2522, 2915, 389, 1690, 1043, 2877, 287, 393, 319, 262, 5920, 286, 5384, 393, 584, 4695, 13, 383, 16586, 3349, 5951, 286, 867, 3108, 15147, 18842, 2522, 2915, 318, 5214, 7200, 34, 357, 4089, 7200, 37, 828, 262, 3487, 1692, 1767, 5951, 13, 14937, 2522, 41896, 20296, 423, 1593, 3544, 287, 2057, 11824, 11, 1390, 9891, 11, 32132, 11, 6099, 290, 8237, 13]
[2061, 2099, 286, 26433, 318, 8811, 973, 287, 11824, 286, 9013, 884, 355, 9891, 290, 32132, 30]
[6880, 2522, 41896, 20296]


In [5]:
# train, test = train_test_split(df, test_size=0.2, random_state=42)
train, dev = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# tokenizer.encode('[PAD]')

In [ ]:
# type(df['support'][0])

In [ ]:
# tokenizer(df['support'][0],padding='max_length')

In [6]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
tokenizer.add_special_tokens({'sep_token': '[SEP]'})
model.resize_token_embeddings(len(tokenizer))


Embedding(50259, 768)

In [8]:
tokenizer.encode('[SEP]')

[50258]

In [9]:
tokenizer.encode('[PAD]')

[50257]

In [7]:
PAD_TOKEN = '[PAD]'
SEP_TOKEN = '[SEP]'

In [8]:
class CustomeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_encoding = self.tokenizer(
            str(row['support']),
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        output_encoding = self.tokenizer(
            '{} {} {}'.format(str(row['question']), SEP_TOKEN,str(row['correct_answer'])),
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        labels = output_encoding['input_ids'].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return dict(
            input_ids=input_encoding['input_ids'].flatten(),
            attention_mask=input_encoding['attention_mask'].flatten(),
            labels=output_encoding['input_ids'].flatten()
        )

In [9]:
max_len = 1024
batch_size = 4

In [10]:
train_dataset = CustomeDataset(train, tokenizer)
# test_dataset = CustomeDataset(test, tokenizer)
dev_dataset = CustomeDataset(dev, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size)

In [11]:
lr = 2e-5
epochs = 1

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=lr)

best_dev_loss = float('inf')
for epoch in range(epochs):
    model.train()
    train_loop = tqdm(train_dataloader, leave=True)
    for batch in train_loop:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        train_loop.set_description(f'Epoch {epoch}')
        train_loop.set_postfix(loss=loss.item())

    model.eval()
    dev_loss = 0
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            dev_loss += outputs.loss.item()

    dev_loss /= len(dev_dataloader)

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        model.save_pretrained("gpt_2_fine_tuning")
        tokenizer.save_pretrained("gpt_2_fine_tuning")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 0: 100%|██████████| 2096/2096 [48:57<00:00,  1.40s/it, loss=0.0789]


In [42]:
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Model config vocab size: {model.config.vocab_size}")


Tokenizer vocab size: 50257
Model config vocab size: 50259


In [13]:
model = AutoModelForCausalLM.from_pretrained("gpt_2_fine_tuning")
tokenizer = AutoTokenizer.from_pretrained("gpt_2_fine_tuning")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

def generate_text(context, max_length=1024):
    inputs = tokenizer.encode_plus(context, return_tensors="pt", max_length=1024, truncation=True).to(device)

    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=1024,
        # num_return_sequences=1,
        # pad_token_id=tokenizer.eos_token_id
        top_p=0.9,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.2,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(context)
    print(generated_text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

model.save_pretrained("gpt_2_fine_tuning")
tokenizer.save_pretrained("gpt_2_fine_tuning")

shutil.make_archive('/content/gpt_2_fine_tuning', 'zip', 'gpt_2_fine_tuning')# zip

files.download('/content/gpt_2_fine_tuning.zip')

KeyboardInterrupt: 

In [14]:
# Test with a shorter input
text = "Green hydrogen is produced using electricity to power an electrolyser that splits hydrogen from water."
generate_text(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Green hydrogen is produced using electricity to power an electrolyser that splits hydrogen from water.
Green hydrogen is produced using electricity to power an electrolyser that splits hydrogen from water.
